# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# Import libraries
import pandas as pd                # For data manipulation
from sqlalchemy import create_engine  # For connecting to SQLite database
from sklearn.model_selection import train_test_split, GridSearchCV, KFold  # For splitting data and hyperparameter tuning
from sklearn.pipeline import Pipeline  # For creating a machine learning pipeline
from sklearn.preprocessing import FunctionTransformer  # For transforming features
from sklearn.multioutput import MultiOutputClassifier  # For multi-output classification
from sklearn.ensemble import RandomForestClassifier  # For the classification model
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression  # For the classification model
from sklearn.metrics import classification_report, accuracy_score  # For evaluation
import warnings

# Download necessary NLTK resources if not already installed
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# load data from database
engine = create_engine('sqlite:///data/InsertDatabaseName.db')
df = pd.read_sql_table("disaster_messages", con=engine)
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    # Step 1: Convert text to lowercase
    text = text.lower()

    # Step 2: Tokenize the text into words
    tokens = word_tokenize(text)

    # Step 3: Remove stopwords (optional)
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # Step 4: Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:

pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidfTransformer', TfidfTransformer()),
    ('classifier', MultiOutputClassifier(RandomForestClassifier()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=40)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
from sklearn.metrics import classification_report

# Iterate through each category in Y
for i, column in enumerate(Y.columns):
    print(f"Category: {column}")
    
    # Check if the model made any positive predictions for this category
    if sum(Y_pred[:, i]) == 0:
        print(f"No positive predictions for {column}. Metrics may be undefined.\n")
        continue  # Skip the classification report for this category to avoid warnings
    
    # Generate and print the classification report for the current category
    print(classification_report(y_test[column], Y_pred[:, i]))
    print("\n")


Category: related
             precision    recall  f1-score   support

          0       0.65      0.43      0.52      1246
          1       0.83      0.93      0.88      3968
          2       0.43      0.10      0.16        30

avg / total       0.79      0.80      0.79      5244



Category: request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4324
          1       0.81      0.43      0.56       920

avg / total       0.88      0.88      0.87      5244



Category: offer
No positive predictions for offer. Metrics may be undefined.

Category: aid_related
             precision    recall  f1-score   support

          0       0.75      0.88      0.81      3067
          1       0.77      0.58      0.66      2177

avg / total       0.76      0.75      0.75      5244



Category: medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0.96      4836
          1       0.62      0.

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7898f4dfbae8>, vocabulary=None)),
  ('tfidfTransformer',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('classifier',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=F

In [17]:
# Adjust the parameter grid for GridSearchCV
parameters = {
    'classifier__estimator__n_estimators': [50, 100]

# Perform GridSearchCV
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=-1, cv=2)

# Fit the GridSearchCV
cv.fit(X_train, y_train)

# View the best parameters
print("Best parameters:", cv.best_params_)


Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] classifier__estimator__n_estimators=50 ..........................
[CV] classifier__estimator__n_estimators=50 ..........................
[CV] classifier__estimator__n_estimators=100 .........................
[CV] classifier__estimator__n_estimators=100 .........................
[CV] ........... classifier__estimator__n_estimators=50, total= 8.3min
[CV] ........... classifier__estimator__n_estimators=50, total= 8.4min
[CV] .......... classifier__estimator__n_estimators=100, total=12.1min
[CV] .......... classifier__estimator__n_estimators=100, total=12.3min


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 13.0min finished


Best parameters: {'classifier__estimator__n_estimators': 100}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
from sklearn.metrics import classification_report

# Iterate through each category in Y
for i, column in enumerate(Y.columns):
    print(f"Category: {column}")
    
    # Check if the model made any positive predictions for this category
    if sum(Y_pred[:, i]) == 0:
        print(f"No positive predictions for {column}. Metrics may be undefined.\n")
        continue  # Skip the classification report for this category to avoid warnings
    
    # Generate and print the classification report for the current category
    print(classification_report(y_test[column], Y_pred[:, i]))
    print("\n")

Category: related
             precision    recall  f1-score   support

          0       0.65      0.43      0.52      1246
          1       0.83      0.93      0.88      3968
          2       0.43      0.10      0.16        30

avg / total       0.79      0.80      0.79      5244



Category: request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4324
          1       0.81      0.43      0.56       920

avg / total       0.88      0.88      0.87      5244



Category: offer
No positive predictions for offer. Metrics may be undefined.

Category: aid_related
             precision    recall  f1-score   support

          0       0.75      0.88      0.81      3067
          1       0.77      0.58      0.66      2177

avg / total       0.76      0.75      0.75      5244



Category: medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0.96      4836
          1       0.62      0.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [21]:
import pickle
filename = 'classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.